In [766]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display
import datetime
import time
import math
import warnings
warnings.filterwarnings("ignore")
import glob

In [767]:
house1 = pd.read_csv('House1.csv')
house1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 406748 entries, 0 to 406747
Data columns (total 9 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Time            406748 non-null  object 
 1   mains_1         406748 non-null  float64
 2   mains_2         406748 non-null  float64
 3   refrigerator_3  406748 non-null  float64
 4   dishwaser_4     406748 non-null  float64
 5   washer_dryer_5  406748 non-null  float64
 6   microwave_6     406748 non-null  float64
 7   washer_dryer_7  406748 non-null  float64
 8   washer_dryer_8  406748 non-null  float64
dtypes: float64(8), object(1)
memory usage: 27.9+ MB


In [768]:
dates = {}
dates = [str(time)[:10] for time in house1.Time.values]
dates = sorted(list(set(dates)))
print('House {0} data contain {1} days from {2} to {3}.'.format(1,len(dates),dates[0], dates[-1]))

House 1 data contain 23 days from 2011-04-18 to 2011-05-24.


In [769]:
house1['washer_dryer'] = house1['washer_dryer_5'] + house1['washer_dryer_7'] + house1['washer_dryer_8']
house1['mains'] = house1['mains_1']+house1['mains_2']
del house1['washer_dryer_5']
del house1['washer_dryer_7']
del house1['washer_dryer_8']
del house1['mains_1']
del house1['mains_2']
house1.head(2)

,Time,refrigerator_3,dishwaser_4,microwave_6,washer_dryer,mains
0,2011-04-18 13:22:13,6.0,0.0,5.0,0.0,341.03
1,2011-04-18 13:22:16,6.0,0.0,5.0,0.0,342.36


# HOUSE 1

In [770]:
from sklearn.cluster import KMeans

### Fridge

In [771]:
fridge = house1['refrigerator_3']
fridge = pd.DataFrame(fridge)

In [772]:
km = KMeans(n_clusters=2)
km.fit(fridge)

KMeans(n_clusters=2)

In [773]:
centroids  = km.cluster_centers_
cent_fridge=centroids
cent_fridge

array([[  6.65272656],
       [201.06403671]])

In [774]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['fridge_label'])

dummy = house1
dummy = dummy.join(labels)

test = dummy[['refrigerator_3','fridge_label']] 

on = test[test['fridge_label'] == 1] ## on status
on = on['refrigerator_3'].std() 

off = test[test['fridge_label'] == 0] ## on status
off = off['refrigerator_3'].std() 

In [775]:
d = off/(on+off)
lambda_fridge = ((1.0-d)*(cent_fridge[0]))+(d*cent_fridge[1])

### Dishwasher (DW)

In [776]:
dw = house1['dishwaser_4']
dw = pd.DataFrame(dw)

In [777]:
km = KMeans(n_clusters=2)
km.fit(dw)

KMeans(n_clusters=2)

In [778]:
centroids  = km.cluster_centers_
cent_dw=centroids
cent_dw

array([[   5.35537745],
       [1085.4334193 ]])

In [779]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['dw_label'])

dummy = house1
dummy = dummy.join(labels)

test = dummy[['dishwaser_4','dw_label']] 

on = test[test['dw_label'] == 1] ## on status
on = on['dishwaser_4'].std() 

off = test[test['dw_label'] == 0] ## off status
off = off['dishwaser_4'].std() 

In [780]:
d = off/(on+off)
lambda_dw = ((1.0-d)*(cent_dw[0]))+(d*cent_dw[1])

### Microwave (MW)

In [781]:
mw = house1['microwave_6']
mw = pd.DataFrame(mw)

In [782]:
km = KMeans(n_clusters=2)
km.fit(mw)

KMeans(n_clusters=2)

In [783]:
centroids  = km.cluster_centers_
cent_mw=centroids
cent_mw

array([[   4.31099539],
       [1520.22549834]])

In [784]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['mw_label'])

dummy = house1
dummy = dummy.join(labels)

test = dummy[['microwave_6','mw_label']] 

on = test[test['mw_label'] == 1] ## on status
on = on['microwave_6'].std() 

off = test[test['mw_label'] == 0] ## on status
off = off['microwave_6'].std() 

In [785]:
d = off/(on+off)
lambda_mw = ((1.0-d)*(cent_mw[0]))+(d*cent_mw[1])

### Washer Dryer

In [786]:
wd = house1['washer_dryer']
wd = pd.DataFrame(wd)

In [787]:
km = KMeans(n_clusters=2)
km.fit(wd)

KMeans(n_clusters=2)

In [788]:
centroids  = km.cluster_centers_
cent_wd=centroids
cent_wd

array([[2.61673342e+00],
       [2.81289642e+03]])

In [789]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['wd_label'])

dummy = house1
dummy = dummy.join(labels)

test = dummy[['washer_dryer','wd_label']] 

on = test[test['wd_label'] == 1] ## on status
on = on['washer_dryer'].std() 

off = test[test['wd_label'] == 0] ## on status
off = off['washer_dryer'].std() 

In [790]:
d = off/(on+off)
lambda_wd = ((1.0-d)*(cent_wd[0]))+(d*cent_wd[1])

## Adding Appliance Status to the Dataframe

In [791]:
house1['fridge_stats'] = house1['refrigerator_3'] - lambda_fridge
house1['fridge_stat'] = [1 if x >= 0 else 0 for x in house1['fridge_stats']]
house1.fridge_stat

0         0
1         0
2         0
3         0
4         0
         ..
406743    1
406744    1
406745    1
406746    1
406747    1
Name: fridge_stat, Length: 406748, dtype: int64

In [792]:
house1['dishwasher_stats'] = house1['dishwaser_4'] - lambda_dw
house1['dishwasher_stat'] = [1 if x >= 0 else 0 for x in house1['dishwasher_stats']]
house1.dishwasher_stat

0         0
1         0
2         0
3         0
4         0
         ..
406743    0
406744    0
406745    0
406746    0
406747    0
Name: dishwasher_stat, Length: 406748, dtype: int64

In [793]:
house1['microwave_stats'] = house1['microwave_6'] - lambda_mw
house1['microwave_stat'] = [1 if x >= 0 else 0 for x in house1['microwave_stats']]
house1.microwave_stat

0         0
1         0
2         0
3         0
4         0
         ..
406743    0
406744    0
406745    0
406746    0
406747    0
Name: microwave_stat, Length: 406748, dtype: int64

In [794]:
house1['washerdryer_stats'] = house1['washer_dryer'] - lambda_wd
house1['washerdryer_stat'] = [1 if x >= 0 else 0 for x in house1['washerdryer_stats']]
house1.washerdryer_stat

0         0
1         0
2         0
3         0
4         0
         ..
406743    0
406744    0
406745    0
406746    0
406747    0
Name: washerdryer_stat, Length: 406748, dtype: int64

In [795]:
del house1['fridge_stats']
del house1['dishwasher_stats']
del house1['microwave_stats']
del house1['washerdryer_stats']

del house1['refrigerator_3']
del house1['dishwaser_4']
del house1['microwave_6']
del house1['washer_dryer']

house1

,Time,mains,fridge_stat,dishwasher_stat,microwave_stat,washerdryer_stat
0,2011-04-18 13:22:13,341.03,0,0,0,0
1,2011-04-18 13:22:16,342.36,0,0,0,0
2,2011-04-18 13:22:20,342.52,0,0,0,0
3,2011-04-18 13:22:23,342.07,0,0,0,0
4,2011-04-18 13:22:26,341.77,0,0,0,0
...,...,...,...,...,...,...
406743,2011-05-24 19:56:20,274.38,1,0,0,0
406744,2011-05-24 19:56:23,273.69,1,0,0,0
406745,2011-05-24 19:56:27,274.07,1,0,0,0
406746,2011-05-24 19:56:30,274.75,1,0,0,0


In [796]:
house1.describe()

,mains,fridge_stat,dishwasher_stat,microwave_stat,washerdryer_stat
count,406748.000000,406748.000000,406748.000000,406748.000000,406748.000000
mean,383.496417,0.254342,0.023260,0.012288,0.015727
std,778.299465,0.435491,0.150729,0.110167,0.124418
min,90.930000,0.000000,0.000000,0.000000,0.000000
25%,140.157500,0.000000,0.000000,0.000000,0.000000
50%,177.050000,0.000000,0.000000,0.000000,0.000000
75%,349.240000,1.000000,0.000000,0.000000,0.000000
max,11926.560000,1.000000,1.000000,1.000000,1.000000


In [797]:
ddf = house1.reset_index() # same as df.reset_index(level=0)
ddf.to_csv('House1 with operational status.csv', index = False) # this is for house 1 of the low frequency REDD dataset

# HOUSE 3

In [798]:
house3 = pd.read_csv('House3.csv')
house3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 376150 entries, 0 to 376149
Data columns (total 8 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Time            376150 non-null  object 
 1   mains_1         376150 non-null  float64
 2   mains_2         376150 non-null  float64
 3   refrigerator_3  376150 non-null  float64
 4   dishwaser_4     376150 non-null  float64
 5   washer_dryer_5  376150 non-null  float64
 6   washer_dryer_6  376150 non-null  float64
 7   microwave_7     376150 non-null  float64
dtypes: float64(7), object(1)
memory usage: 23.0+ MB


In [799]:
dates = {}
dates = [str(time)[:10] for time in house1.Time.values]
dates = sorted(list(set(dates)))
print('House {0} data contain {1} days from {2} to {3}.'.format(1,len(dates),dates[0], dates[-1]))

House 1 data contain 23 days from 2011-04-18 to 2011-05-24.


In [800]:
house3['washer_dryer'] = house3['washer_dryer_5'] + house3['washer_dryer_6']
house3['mains'] = house3['mains_2'] + house3['mains_1']
del house3['washer_dryer_5']
del house3['washer_dryer_6']
del house3['mains_1']
del house3['mains_2']
house3

,Time,refrigerator_3,dishwaser_4,microwave_7,washer_dryer,mains
0,2011-04-16 05:11:30,119.0,0.0,2.0,0.0,758.64
1,2011-04-16 05:11:33,118.0,1.0,2.0,0.0,758.13
2,2011-04-16 05:11:36,119.0,0.0,2.0,0.0,759.88
3,2011-04-16 05:11:40,119.0,1.0,2.0,0.0,759.88
4,2011-04-16 05:11:43,117.0,0.0,2.0,0.0,755.87
...,...,...,...,...,...,...
376145,2011-05-31 00:19:23,8.0,3.0,1.0,0.0,17.70
376146,2011-05-31 00:19:26,8.0,3.0,2.0,0.0,17.71
376147,2011-05-31 00:19:30,8.0,3.0,2.0,0.0,17.68
376148,2011-05-31 00:19:33,8.0,3.0,2.0,0.0,17.65


In [801]:
from sklearn.cluster import KMeans

In [802]:
house3.describe()

,refrigerator_3,dishwaser_4,microwave_7,washer_dryer,mains
count,376150.000000,376150.000000,376150.000000,376150.000000,376150.000000
mean,47.659705,7.365479,8.422329,81.368424,445.189291
std,68.182648,65.860263,102.046689,592.545013,731.694496
min,0.000000,0.000000,0.000000,0.000000,1.400000
25%,0.000000,1.000000,2.000000,0.000000,142.430000
50%,1.000000,1.000000,2.000000,0.000000,252.140000
75%,116.000000,1.000000,2.000000,0.000000,453.980000
max,1582.000000,786.000000,1817.000000,5660.000000,8059.920000


### Fridge

In [803]:
fridge = house3['refrigerator_3']
fridge = pd.DataFrame(fridge)

In [804]:
km = KMeans(n_clusters=2)
km.fit(fridge)

KMeans(n_clusters=2)

In [805]:
centroids  = km.cluster_centers_
cent_fridge=centroids
cent_fridge

array([[126.49792431],
       [  0.43144457]])

In [806]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['fridge_label'])

dummy = house3
dummy = dummy.join(labels)

test = dummy[['refrigerator_3','fridge_label']] 

on = test[test['fridge_label'] == 1] ## on status
on = on['refrigerator_3'].std() 

off = test[test['fridge_label'] == 0] ## on status
off = off['refrigerator_3'].std() 

In [807]:
d = off/(on+off)
lambda_fridge = ((1.0-d)*(cent_fridge[0]))+(d*cent_fridge[1])

### Dishwasher (DW)

In [808]:
dw = house3['dishwaser_4']
dw = pd.DataFrame(dw)

In [809]:
km = KMeans(n_clusters=2)
km.fit(dw)

KMeans(n_clusters=2)

In [810]:
centroids  = km.cluster_centers_
cent_dw=centroids
cent_dw

array([[  1.62862425],
       [736.86473595]])

In [811]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['dw_label'])

dummy = house3
dummy = dummy.join(labels)

test = dummy[['dishwaser_4','dw_label']] 

on = test[test['dw_label'] == 1] ## on status
on = on['dishwaser_4'].std() 

off = test[test['dw_label'] == 0] ## on status
off = off['dishwaser_4'].std() 

In [812]:
d = off/(on+off)
lambda_dw = ((1.0-d)*(cent_dw[0]))+(d*cent_dw[1])

### Microwave (MW)

In [813]:
mw = house3['microwave_7']
mw = pd.DataFrame(mw)

In [814]:
km = KMeans(n_clusters=2)
km.fit(mw)

KMeans(n_clusters=2)

In [815]:
centroids  = km.cluster_centers_
cent_mw=centroids
cent_mw

array([[   2.36282749],
       [1716.10827068]])

In [816]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['mw_label'])

dummy = house3
dummy = dummy.join(labels)

test = dummy[['microwave_7','mw_label']] 

on = test[test['mw_label'] == 1] ## on status
on = on['microwave_7'].std() 

off = test[test['mw_label'] == 0] ## on status
off = off['microwave_7'].std() 

In [817]:
d = off/(on+off)
lambda_mw = ((1.0-d)*(cent_mw[0]))+(d*cent_mw[1])

### Washer Dryer

In [818]:
wd = house3['washer_dryer']
wd = pd.DataFrame(wd)

In [819]:
km = KMeans(n_clusters=2)
km.fit(wd)

KMeans(n_clusters=2)

In [820]:
centroids  = km.cluster_centers_
cent_wd=centroids
cent_wd

array([[   7.13361149],
       [4772.22312287]])

In [821]:
labels = km.labels_
labels = pd.DataFrame (labels, columns = ['wd_label'])

dummy = house3
dummy = dummy.join(labels)

test = dummy[['washer_dryer','wd_label']] 

on = test[test['wd_label'] == 1] ## on status
on = on['washer_dryer'].std() 

off = test[test['wd_label'] == 0] ## off status
off = off['washer_dryer'].std() 

In [822]:
d = off/(on+off)
lambda_wd = ((1.0-d)*(cent_wd[0]))+(d*cent_wd[1])

## Adding Appliance Status to the Dataframe

In [823]:
house3['fridge_stats'] = house3['refrigerator_3'] - lambda_fridge
house3['fridge_stat'] = [1 if x >= 0 else 0 for x in house3['fridge_stats']]
house3.fridge_stat

0         1
1         1
2         1
3         1
4         1
         ..
376145    1
376146    1
376147    1
376148    1
376149    1
Name: fridge_stat, Length: 376150, dtype: int64

In [824]:
house3['dishwasher_stats'] = house3['dishwaser_4'] - lambda_dw
house3['dishwasher_stat'] = [1 if x >= 0 else 0 for x in house3['dishwasher_stats']]
house3.dishwasher_stat

0         0
1         0
2         0
3         0
4         0
         ..
376145    0
376146    0
376147    0
376148    0
376149    0
Name: dishwasher_stat, Length: 376150, dtype: int64

In [825]:
house3['microwave_stats'] = house3['microwave_7'] - lambda_mw
house3['microwave_stat'] = [1 if x >= 0 else 0 for x in house3['microwave_stats']]
house3.microwave_stat

0         0
1         0
2         0
3         0
4         0
         ..
376145    0
376146    0
376147    0
376148    0
376149    0
Name: microwave_stat, Length: 376150, dtype: int64

In [826]:
house3['washerdryer_stats'] = house3['washer_dryer'] - lambda_wd
house3['washerdryer_stat'] = [1 if x >= 0 else 0 for x in house3['washerdryer_stats']]
house3.washerdryer_stat

0         0
1         0
2         0
3         0
4         0
         ..
376145    0
376146    0
376147    0
376148    0
376149    0
Name: washerdryer_stat, Length: 376150, dtype: int64

In [827]:
del house3['fridge_stats']
del house3['dishwasher_stats']
del house3['microwave_stats']
del house3['washerdryer_stats']

del house3['refrigerator_3']
del house3['dishwaser_4']
del house3['microwave_7']
del house3['washer_dryer']

In [828]:
house3

,Time,mains,fridge_stat,dishwasher_stat,microwave_stat,washerdryer_stat
0,2011-04-16 05:11:30,758.64,1,0,0,0
1,2011-04-16 05:11:33,758.13,1,0,0,0
2,2011-04-16 05:11:36,759.88,1,0,0,0
3,2011-04-16 05:11:40,759.88,1,0,0,0
4,2011-04-16 05:11:43,755.87,1,0,0,0
...,...,...,...,...,...,...
376145,2011-05-31 00:19:23,17.70,1,0,0,0
376146,2011-05-31 00:19:26,17.71,1,0,0,0
376147,2011-05-31 00:19:30,17.68,1,0,0,0
376148,2011-05-31 00:19:33,17.65,1,0,0,0


In [829]:
house3.describe()

,mains,fridge_stat,dishwasher_stat,microwave_stat,washerdryer_stat
count,376150.000000,376150.000000,376150.000000,376150.000000,376150.000000
mean,445.189291,0.377230,0.007803,0.003693,0.015595
std,731.694496,0.484694,0.087988,0.060655,0.123902
min,1.400000,0.000000,0.000000,0.000000,0.000000
25%,142.430000,0.000000,0.000000,0.000000,0.000000
50%,252.140000,0.000000,0.000000,0.000000,0.000000
75%,453.980000,1.000000,0.000000,0.000000,0.000000
max,8059.920000,1.000000,1.000000,1.000000,1.000000


In [830]:
ddf = house3.reset_index() # same as df.reset_index(level=0)
ddf.to_csv('House3 with operational status.csv', index = False) # this is for house 1 of the low frequency REDD dataset